In [1]:
from pandas import DataFrame as df
from pandas import read_csv
TrainDataset = read_csv('./train.csv')
TrainDataset= df(TrainDataset)
TrainDataset=TrainDataset.dropna()

TrainDataset=TrainDataset.drop(['Name','Ticket','Cabin'],axis=1)
TrainDataset["FamilySize"] = TrainDataset["SibSp"] + TrainDataset["Parch"] + 1




In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Age', 'Fare', 'FamilySize']),
        ('cat', OneHotEncoder(handle_unknown='ignore'), ['Pclass', 'Sex', 'Embarked'])
    ])


base_models_pipeline = [
  ('random_forest', Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42, n_estimators=1000))
  ])),
  ('gradient_boosting', Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(random_state=42, n_estimators=1000))
  ]))
]

stacking_pipeline = StackingClassifier(
  estimators=base_models_pipeline,
  final_estimator=LogisticRegression(),
  cv=5
)


X = TrainDataset.drop(columns=['Survived', 'PassengerId'])
y = TrainDataset['Survived']

stacking_pipeline.fit(X, y)
y_pred_stacking = stacking_pipeline.predict(X)


stacking_accuracy = accuracy_score(y, y_pred_stacking)

print(f'Stacking accuracy: {stacking_accuracy:.2f}')

submission=pd.DataFrame({'PassengerId':TrainDataset['PassengerId'],'Survived':y_pred_stacking})
submission.to_csv('Try3.csv',index=False,header=True)






Stacking accuracy: 1.00
